This notebook is in contination to GNNv2(https://www.kaggle.com/code/mayankk9/gnnv2) notebook which has codes for generating customer and articles embedding using graph neural netowrk. 
Here in the notebook, which is really unstructured and commented out, I have tried a transformer architecture with context included (customer embedding) with encoder output and fed to decoder to predict sequences of articles purchased. Unfortunatelly, the model could not show any sign of convergences and got stuck. If you believe in the architecture or want a more organized notebook, please comment and I will post a more organized version. 
The notebook has Transformers source code to handle masking with a small value rather than making it nan. 

In [ ]:


! nvcc --version; python --version
! pip install dgl-cu110 dglgo -f https://data.dgl.ai/wheels/repo.html



In [ ]:


import pandas as pd
pd.set_option("display.max_columns", 100)
import cudf
import numpy as np
import os
import dgl
import torch
import cupy as cp
from cuml.preprocessing.LabelEncoder import LabelEncoder
from cuml.preprocessing.TargetEncoder import TargetEncoder
import joblib
import tqdm

In [ ]:
# ! zip -r d.zip ../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv

In [ ]:


feat = {'customers':{'cat':['club_member_status', 'fashion_news_frequency', 'postal_code'],
       'cont':['age']}, 
        'articles': {'cat': ['product_code', 'product_type_no', 'product_group_name',
              'graphical_appearance_no', 'colour_group_code','perceived_colour_master_id' , 'garment_group_no',
              'section_no', 'index_group_no'],
        'cont':[]}
       }



In [ ]:
import dgl
from dgl.data import DGLDataset
import torch
import os
import numpy as np
import cudf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
wmode = False

In [ ]:
!pip install wandb -qqq
import wandb
import os
os.environ["WANDB_API_KEY"] = "7e40122ef4016b01cdc4a60bcd7ee7a251b020a7"
if wmode: wandb.init(project="HnmRGCNv2")

In [ ]:
import dgl
from dgl.data import DGLDataset
import torch
import os
import numpy as np
import cudf
from sklearn.preprocessing import MinMaxScaler

class HandMData(DGLDataset):
    def __init__(self):
        super().__init__(name='HandMData')

    def process(self):
        global wmode
        base = "../input/h-and-m-personalized-fashion-recommendations/"
        
        nodes_articles = cudf.read_csv(f'./{base}/articles.csv')
        nodes_articles['article_id'] = nodes_articles['article_id'].astype('int32')
        le_a = LabelEncoder(handle_unknown='ignore')
        le_a.fit(nodes_articles['article_id'])
        
        import pickle
        import joblib

        joblib.dump(le_a, 'label_encoder.joblib')
#         label_encoder = joblib.load('label_encoder.joblib')
        with open("le_a.pkl", "wb") as fp:
            pickle.dump(le_a, fp)
        if wmode: wandb.save(f"le_a.pkl")
        nodes_articles['article_id'] = le_a.transform(nodes_articles['article_id'])
        del le_a
        nodes_articles = nodes_articles.sort_values('article_id')
        
        for col in feat['articles']['cat']:
            le_a = LabelEncoder(handle_unknown='ignore')
            le_a.fit(nodes_articles[col])
#             print(le_a.classes_)
            import pickle
            with open(f"le_{col}.pkl", "wb") as fp:
                pickle.dump(le_a, fp)
            nodes_articles[col] = le_a.transform(nodes_articles[col])
            del le_a
        
#         le_ap = LabelEncoder(handle_unknown='ignore')
#         le_ap.fit(nodes_articles['product_code'].fillna('PC'))
#         nodes_articles['product_code'] = le_ap.transform(nodes_articles['product_code'])
#         with open("le_ap.pkl", "wb") as fp:
#             pickle.dump(le_ap, fp)
#         del le_ap
#         del nodes_articles
        
        nodes_customers = cudf.read_csv(f'./{base}/customers.csv')
        nodes_customers['customer_id'] = nodes_customers['customer_id'].str[-16:].str.hex_to_int().astype('int64')
        
        le_c = LabelEncoder(handle_unknown='ignore')
        le_c.fit(nodes_customers['customer_id'])
        nodes_customers['customer_id'] = le_c.transform(nodes_customers['customer_id'])
        import pickle
        with open("le_c.pkl", "wb") as fp:
            pickle.dump(le_c, fp)
        if wmode: wandb.save(f"le_c.pkl")
        del le_c
        nodes_customers = nodes_customers.sort_values('customer_id')
        # toodo include age at end MinMax Scaler
        for col in feat['customers']['cat']:
            le_a = LabelEncoder(handle_unknown='ignore')
            le_a.fit(nodes_customers[col])
#             print(le_a.classes_)
            import pickle
            with open(f"le_{col}.pkl", "wb") as fp:
                pickle.dump(le_a, fp)
            if wmode: wandb.save(f"le_{col}.pkl")
            nodes_customers[col] = le_a.transform(nodes_customers[col])
            del le_a
        
        scaler = MinMaxScaler()
        scaler.fit_transform(np.asarray(nodes_customers['age'].to_arrow()).reshape(-1,1))
        with open(f"MMxS_age.pkl", "wb") as fp:
            pickle.dump(scaler, fp)
        if wmode: wandb.save(f"MMxS_age.pkl")
        transaction_df = cudf.read_csv(f'{base}/transactions_train.csv')
        transaction_df['customer_id'] = transaction_df['customer_id'].str[-16:].str.hex_to_int().astype('int64')
        transaction_df['article_id'] = transaction_df['article_id'].astype('int32')
        
        with open("le_c.pkl", "rb") as fp:
            le_c = pickle.load(fp)
        
        transaction_df['customer_id'] = le_c.transform(transaction_df['customer_id'])
        del le_c #todo del unkown from here may be
        
        with open("le_a.pkl", "rb") as fp:
            le_a = pickle.load(fp)
        transaction_df['article_id'] = le_a.transform(transaction_df['article_id'])
        del le_a
        
        edges_data = transaction_df.groupby(['customer_id', 'article_id'])['t_dat'].count()
        del transaction_df
        
        edges_data = edges_data.reset_index().rename(columns={'t_dat':'count'})
        edge_features = torch.ones(edges_data.shape[0], dtype=torch.int32)#torch.from_numpy(cp.asnumpy(edges_data['count']))
        #todo change edge_feature to make it regression link 
        graph_data = {
        ('customers', 'buys', 'articles'): (torch.from_numpy(cp.asnumpy(edges_data['customer_id'].astype('int32'))),
                                          torch.from_numpy(cp.asnumpy(edges_data['article_id'].astype('int32')))),
        ('articles', 'boughtby', 'customers'): (torch.from_numpy(cp.asnumpy(edges_data['article_id'].astype('int32'))),
                                             torch.from_numpy(cp.asnumpy(edges_data['customer_id'].astype('int32'))))}
        
        
        del edges_data
        # todo all doubles any help include feature for edges same torch float32 [[feat_x]]
        self.g = dgl.heterograph(graph_data)
        self.g.edges['buys'].data['u'] = edge_features
        self.g.edges['boughtby'].data['u'] = edge_features
        del edge_features
        
        cust_cols = feat['customers']['cat'] + feat['customers']['cont']
        art_cols = feat['articles']['cat'] + feat['articles']['cont']
        nodes_customers[cust_cols].fillna(0, inplace=True)
        nodes_articles[art_cols].fillna(0, inplace=True)
        
        self.g.nodes['customers'].data['h'] = torch.from_numpy(nodes_customers[cust_cols].as_matrix().astype('float32'))
        self.g.nodes['customers'].data['c'] = torch.ones(nodes_customers.shape[0])
#         print(nodes_articles[art_cols].as_matrix())
        self.g.nodes['articles'].data['h'] = torch.from_numpy(nodes_articles[art_cols].as_matrix().astype('float32'))[:-2]
        self.g.nodes['articles'].data['c'] = torch.zeros(nodes_articles.shape[0])[:-2]
        
        del nodes_customers
        del nodes_articles
#         self.graph = dgl.graph((edges_src, edges_dst), num_nodes=nodes_data.shape[0])
#         self.graph.ndata['feat'] = node_features
#         self.graph.ndata['label'] = node_labels
#         self.graph.edata['weight'] = edge_features

        # If your dataset is a node classification dataset, you will need to assign
        # masks indicating whether a node belongs to training, validation, and test set.
        for sub_graph in ['customers', 'articles']:
            n_nodes = self.g.num_nodes(f'{sub_graph}')
            n_train = int(n_nodes * 0.7)
            n_val = int(n_nodes * 0.15)
            train_mask = torch.zeros(n_nodes, dtype=torch.bool)
            val_mask = torch.zeros(n_nodes, dtype=torch.bool)
            test_mask = torch.zeros(n_nodes, dtype=torch.bool)
            train_mask[:n_train] = True
            val_mask[n_train:n_train + n_val] = True
            test_mask[n_train + n_val:] = True
            self.g.nodes[f'{sub_graph}'].data['train_mask'] = train_mask
            self.g.nodes[f'{sub_graph}'].data['val_mask'] = val_mask
            self.g.nodes[f'{sub_graph}'].data['test_mask'] = test_mask
        
        for etype in self.g.etypes:
            n_edges = self.g.edges(etype=etype, form='eid').shape[0]
            n_train = int(n_edges * 0.7)
            n_val = int(n_edges * 0.15)
            train_mask = torch.zeros(n_edges, dtype=torch.bool)
            val_mask = torch.zeros(n_edges, dtype=torch.bool)
            test_mask = torch.zeros(n_edges, dtype=torch.bool)
            train_mask[:n_train] = True
            val_mask[n_train:n_train + n_val] = True
            test_mask[n_train + n_val:] = True
            self.g.edges[f'{etype}'].data['train_mask'] = train_mask
            self.g.edges[f'{etype}'].data['val_mask'] = val_mask
            self.g.edges[f'{etype}'].data['test_mask'] = test_mask
        self.g = self.g.to(torch.device('cuda'))
    def __getitem__(self, i):
        return self.g

    def __len__(self):
        return 1

In [ ]:
# dataset = HandMData()
graph = HandMData()[0]
print(graph)

In [ ]:
# # definite infoleak as other set truth can be used as ng 
# train_eid_dict = {etype: (graph.edges[etype].data['train_mask'] == True).nonzero(as_tuple=True)[0] for etype in graph.etypes}
# val_eid_dict   = {etype: (graph.edges[etype].data['val_mask'] == True).nonzero(as_tuple=True)[0] for etype in graph.etypes}
# test_eid_dict   = {etype: (graph.edges[etype].data['test_mask'] == True).nonzero(as_tuple=True)[0] for etype in graph.etypes}

In [ ]:
del graph

In [ ]:


# train_eid_dict = {k:v.to(torch.int32) for k,v in train_eid_dict.items()}
# val_eid_dict = {k:v.to(torch.int32) for k,v in val_eid_dict.items()}
# test_eid_dict = {k:v.to(torch.int32) for k,v in test_eid_dict.items()}



In [ ]:
# edge_sampler = dgl.dataloading.MultiLayerNeighborSampler([15, 10, 5])
# negative_sampler=dgl.dataloading.negative_sampler.Uniform(5)

# sampler = dgl.dataloading.as_edge_prediction_sampler(edge_sampler, exclude='reverse_types',
#                                                      reverse_etypes={'buys': 'boughtby', 'boughtby': 'buys'},
#                                                      negative_sampler= negative_sampler)
                                                     
# train_dataloader = dgl.dataloading.DataLoader(graph, train_eid_dict, sampler, 
#                                         batch_size=1024, shuffle=True, drop_last=False, num_workers=0)
# val_dataloader = dgl.dataloading.DataLoader(graph, val_eid_dict, sampler, 
#                                         batch_size=1024, shuffle=True, drop_last=False, num_workers=0)
# test_dataloader = dgl.dataloading.DataLoader(graph, test_eid_dict, sampler, 
#                                         batch_size=1024, shuffle=True, drop_last=False, num_workers=0)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.nn.pytorch as dglnn
import torch as th

In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F
# import dgl.nn.pytorch as dglnn
# import torch as th

# class StochasticThreeLayerRGCN(nn.Module):
#     def __init__(self, in_feat_dict, hidden_feat, out_feat, canonical_etypes):
#         super().__init__()

#         self.conv1 = dglnn.HeteroGraphConv({
#                 # you can change in_feat to something like in_feat[ntype] here
#                 etype : dglnn.GraphConv(in_feat_dict[utype], hidden_feat, norm='right')
#                 for utype, etype, vtype in canonical_etypes
#                 })
#         self.conv2 = dglnn.HeteroGraphConv({
#                 etype : dglnn.GraphConv(hidden_feat, out_feat, norm='right')
#                 for _, etype, _ in canonical_etypes
#                 })
#         self.conv3 = dglnn.HeteroGraphConv({
#                 etype : dglnn.GraphConv(out_feat, out_feat, norm='right')
#                 for _, etype, _ in canonical_etypes
#                 })
#     def forward(self, blocks, inputs):

#         x1 = self.conv1(blocks[0], inputs)
#         x2 = self.conv2(blocks[1], x1)
#         x3 = self.conv3(blocks[2], x2)
#         return x3

# class HeteroScorePredictor(nn.Module):        
#     def forward(self, edge_subgraph, x):
#         with edge_subgraph.local_scope():
# #             print(edge_subgraph.ndata)
#             edge_subgraph.ndata['h'] = x
#             for etype in edge_subgraph.canonical_etypes:
#                 edge_subgraph.apply_edges(
#                     dgl.function.u_dot_v('h', 'h', 'score'), etype=etype)
#             return edge_subgraph.edata['score']

# class Model(nn.Module):
#     def __init__(self, in_features, hidden_features, out_features,canonical_etypes):
#         super().__init__()
#         self.rgcn = StochasticThreeLayerRGCN(in_features, hidden_features, out_features, canonical_etypes)
#         self.pred = HeteroScorePredictor()
    
#     def _fetch_embLayer(self):
#         return self.rgcn
    
#     def forward(self, graphs, positive_graph, negative_graph, blocks, x):
#         x = self.rgcn(blocks, x)
#         pos_score = self.pred(positive_graph, x)
#         neg_score = self.pred(negative_graph, x)
#         return x, pos_score, neg_score

In [ ]:
# def compute_loss(pos_score, neg_score, canonical_etypes):
#     # Margin loss
#     all_losses = []
#     for given_type in canonical_etypes:
#         n_edges = pos_score[given_type].shape[0]
#         if n_edges == 0:
#             continue
#         all_losses.append((1 - neg_score[given_type].view(n_edges, -1) + pos_score[given_type].unsqueeze(1)).clamp(min=0).mean())
#     return torch.stack(all_losses, dim=0).mean()

# from sklearn.metrics import roc_auc_score
# import tensorflow as tf
# def compute_acc(pos_score):
#     crt = 0
#     t = 0
# #     ruc = 0 
#     for k,v in pos_score.items():
#         y_pred = pos_score[k].cpu().detach().numpy()
#         m = tf.keras.metrics.BinaryAccuracy()
#         y_true = torch.from_numpy(np.ones(y_pred.shape[0]))
#         m.update_state(y_pred, y_true)
#         crt += m.result().numpy()
#         t += y_pred.shape[0]
# #         ruc +=  roc_auc_score(y_true,y_pred)
#     return crt/t

In [ ]:
# def run_train_epoch(train_dataloader):
#     ac_ = 0
#     model.train()
#     with tqdm.tqdm(train_dataloader) as tq:
#         for step, (input_nodes, positive_graph, negative_graph, blocks) in enumerate(tq):
            
#             blocks = [b.to(torch.device('cuda')) for b in blocks]

#             positive_graph = positive_graph.to(torch.device('cuda'))
#             negative_graph = negative_graph.to(torch.device('cuda'))

#             node_features = {'articles': blocks[0].srcdata['h']['articles'], 
#                              'customers': blocks[0].srcdata['h']['customers']}
#             pos_score, neg_score = model(graph, positive_graph, negative_graph, blocks, node_features)
#             loss = compute_loss(pos_score, neg_score, graph.canonical_etypes)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             tq.set_postfix({'loss': '%.03f' % loss.item()}, refresh=False)
            
#             if step % 1000==0:
#                 torch.save(model.state_dict(), f'mb-{step}-train-{loss}-{best_model_path}')
#                 lm = f'mb-{step}-train-{loss}-{best_model_path}'
#                 wandb.save( lm, base_path="./", policy="now")
#             if step % 5000==0:
#                 for fnm in os.listdir():
#                     if fnm.endswith('.pt'):
#                         if fnm == lm:continue
#                         os.remove(fnm)
#             t = compute_acc(pos_score)          
#             wandb.log({"train-loss-mb": loss})
#             wandb.log({"train-acc-mb": t})
#             ac_ += t
            
            
        
        
#         wandb.log({"train-acc-epochs": ac_})
#         print("Accuracy after epoch End on Train ", ac_)
#         torch.save(model.state_dict(), f'epoch-train-{loss}-{best_model_path}')
#         wandb.save( f'epoch-train-{loss}-{best_model_path}')
        
#         return loss, ac_

In [ ]:
# ### function below ###
# best_accuracy = 0
# best_model_path = 'model.pt'


# def run_eval_epoch(train_dataloader):
#     global best_accuracy
#     global best_model_path
#     ac_ = 0
#     model.eval()
#     with tqdm.tqdm(train_dataloader) as tq:
#         for step, (input_nodes, positive_graph, negative_graph, blocks) in enumerate(tq):
#             blocks = [b.to(torch.device('cuda')) for b in blocks]

#             positive_graph = positive_graph.to(torch.device('cuda'))
#             negative_graph = negative_graph.to(torch.device('cuda'))

#             node_features = {'articles': blocks[0].srcdata['h']['articles'], 
#                              'customers': blocks[0].srcdata['h']['customers']}
#             with torch.no_grad():
#                 pos_score, neg_score = model(graph, positive_graph, negative_graph, blocks, node_features)
#                 ac = compute_acc(pos_score) # batch normalized already
#                 if best_accuracy < ac:
#                     best_accuracy = ac
#                     torch.save(model.state_dict(), f'bm-{step}-val-{ac}-{best_model_path}')
#                     wandb.save( f'bm-{step}-val-{ac}-{best_model_path}',base_path="./", policy="now")
#                 loss = compute_loss(pos_score, neg_score, graph.canonical_etypes)
#                 wandb.log({"val-loss-mb": loss})
#                 tq.set_postfix({'loss': '%.03f' % loss.item()}, refresh=False)
#                 t = compute_acc(pos_score)
#                 ac_ += t
#                 wandb.log({"val-acc-mb": t})
                
#         wandb.log({"val-acc-epochs": ac_})
        
#         print("Accuracy after epoch End on Val ", ac_)
#         return loss, ac_

# Training Code

In [ ]:
# wandb.init(project="HnmRGCNv2")
# model = Model(in_features={'customers':4, 'articles':9},
#               hidden_features=512, out_features=256, canonical_etypes=graph.canonical_etypes)
# model = model.cuda()
# optimizer = torch.optim.Adam(model.parameters())
# import time
# wandb.watch(model)
          
    
# for epoch in range(4):
#     train_loss, train_accuracy  = run_train_epoch(train_dataloader) # last batch loss and acc
    
#     eval_loss, eval_accuracy = run_eval_epoch(val_dataloader)    
#     print(f'For epoch {epoch}: {eval_loss, eval_accuracy}')
    
    
# torch.save(x, 'tensor.pt')  
# torch.save(model.state_dict(), f'EOF-train-{best_model_path}')
# wandb.save(f'EOF-train-{best_model_path}') 

# Load Model

In [ ]:
# last_model = wandb.restore('EOF-train-model.pt', 
#                            run_path="mayankk-om-dev/HnmRGCNv2/1g06378a")

# # # use the "name" attribute of the returned object if your framework expects a filename, e.g. as in Keras
# checkpoint = torch.load(last_model.name)

# model = Model(in_features={'customers':4, 'articles':9},
#               hidden_features=512, out_features=256, canonical_etypes=graph.canonical_etypes)
# model = model.cuda()
# model.load_state_dict(checkpoint)

In [ ]:
# graph

In [ ]:
# graph.number_of_nodes("articles")

# Dump Code Dont run load 

In [ ]:
# def dump_embs(model, dataloader, embs_c, embs_a):#, embs_c, embs_a, dim_size):
#     t0 = time.time()
    
#     model.eval()
#     with tqdm.tqdm(dataloader) as tq:
#         for step, (input_nodes, positive_graph, negative_graph, blocks) in enumerate(tq):
#             blocks = [b.to(torch.device('cuda')) for b in blocks]

#             positive_graph = positive_graph.to(torch.device('cuda'))
            
#             node_features = {'articles': blocks[0].srcdata['h']['articles'], 
#                              'customers': blocks[0].srcdata['h']['customers']}
#             with torch.no_grad():
#                 emb_feats, pos_score, neg_score = model(graph, positive_graph, negative_graph, blocks, node_features)
#                 embs_c[blocks[-1].dstdata['_ID']['customers'].long()] = emb_feats['customers'].cpu()
#                 embs_a[blocks[-1].dstdata['_ID']['articles'].long()] = emb_feats['articles'].cpu()
# #             print(positive_graph.nodes['customers'].data['_ID'])
# #             print(positive_graph.nodes['customers'].data['_ID'].shape)
# #             print(blocks[-1])
# #             print(type(blocks[-1]))
# #             print(blocks[-1].dstdata['_ID']['customers'])
# #             print(blocks[-1].dstdata['_ID']['customers'].shape)
# #             print(type(x))
# #             print(x['customers'].shape)
# #             print(input_nodes[:5])
# #             break
#     print(f"took {time.time()-t0} seconds")
#     return embs_c, embs_a       

In [ ]:
# c = torch.empty(N, seq_len, 256)
# emb = torch.randn(N, 256)
# B = A.unsqueeze(1).repeat(1, K, 1)
# import einops
# einops.repeat(x, 'm n -> m k n', k=K)

In [ ]:
# import tqdm
# embs_c = torch.empty(graph.number_of_nodes("customers"), 256)
# embs_a = torch.empty(graph.number_of_nodes("articles"), 256)

# embs_c, embs_a  = dump_embs(model, train_dataloader, embs_c, embs_a)
# embs_c, embs_a  = dump_embs(model, val_dataloader, embs_c, embs_a)
# embs_c, embs_a  = dump_embs(model, test_dataloader, embs_c, embs_a)

In [ ]:
# wandb.init(project="HnmRGCNv2")
# torch.save(embs_c, 'customerGNNEmb.pt')
# wandb.save('customerGNNEmb.pt') 
# torch.save(embs_a, 'articleGNNEmb.pt')  
# wandb.save('articleGNNEmb.pt') 

In [ ]:
# embs_cD = wandb.restore('customerGNNEmb.pt', 
#                            run_path="mayankk-om-dev/HnmRGCNv2/2oooot9x")

# # # use the "name" attribute of the returned object if your framework expects a filename, e.g. as in Keras
# embs_c = torch.load(embs_cD)
# del embs_cD
# embs_aD = wandb.restore('articleGNNEmb.pt', 
#                            run_path="mayankk-om-dev/HnmRGCNv2/2oooot9x")
# embs_a = torch.load(embs_aD)
# del embs_aD

In [ ]:
def load_wandb(file, run_path):
    embs_cD = wandb.restore(file, run_path=run_path)

    # # use the "name" attribute of the returned object if your framework expects a filename, e.g. as in Keras
    embs_c = torch.load(embs_cD.name)
    del embs_cD
    return embs_c

In [ ]:
# No need to dump shallow embdeing use as model layer
# look at how long sequnece transaction sequence are... way long than RNN(Only Attention) 
# not possible else RNN+Attention 

# Transformers Baby for Seq with Condition

# New Architect
Input -  Articles[:10] Warm start encoder (10)
New Signal - Customer Emb Vect + Ecoded --> Encoded 
Pass New Signal to Decoder Block
Outputs -  Articles[10:]


# Custom Transformer to make context aware

## End-To-End Transformer

In [ ]:
###hollow

# Prepare Seq Data

In [ ]:
import pickle
pkl_loc = '.'
with open(f'{pkl_loc}/le_a.pkl', 'rb') as fp:
    le_a = pickle.load(fp)

with open(f'{pkl_loc}/le_c.pkl', 'rb') as fp:
    le_c = pickle.load(fp)
le_a.classes_ = le_a.classes_[:-2]

# le_ad = wandb.restore("le_a.pkl", "mayankk-om-dev/HnmRGCNv2/34r1klbo")
# le_a = pickle.load(le_ad.name)

# le_cd = wandb.restore("le_c.pkl", "mayankk-om-dev/HnmRGCNv2/34r1klbo")
# le_c = pickle.load(le_cd.name)

# le_a.classes_ = le_a.classes_[:-2]

In [ ]:
# import gc
# gc.collect()

In [ ]:
# transF = "../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv"
# df_trans = cudf.read_csv(transF)
# df_trans["customer_id"] = df_trans['customer_id'].str[-16:].str.hex_to_int().astype('int64')
# df_trans['article_id'] = df_trans['article_id'].astype('int32')
# df_trans['customer_id'] = le_c.transform(df_trans['customer_id'])
# df_trans['article_id'] = le_a.transform(df_trans['article_id'])
# df_trans = df_trans.to_pandas()
# grp = df_trans.sort_values(['customer_id','t_dat']).groupby('customer_id')
# gdd =grp['article_id'].apply(list)
# gdd = gdd.reset_index()

In [ ]:
# gdd.head()
# # transform 

In [ ]:
# d = {'customer_id': [], 'article_src':[], 'article_dst':[]}

# def split_row(row):
# #     print(type(row['article_dst']))
#     len_ = len(row['article_dst'])
#     cust = row['customer_id']
#     global d
#     if len_<11:
#         return row['article_dst']
#     else:
#         ret = row['article_dst'][:10] # max_seqsize =10
#         extras = row['article_dst'][10:]
#     while len(extras)>=15:
#         # add to other df
#         add = extras[:15]
#         d['customer_id'].append(cust)
#         d['article_src'].append(add[:5])
#         d['article_dst'].append(add[5:])
#         extras = extras[15:]
    
#     # add extras if extras and greater than 5
# #     if extras and len(extras)>5:
# #         d['customer_id'].append(cust)
# #         d['article_src'].append(extras[:5])
# #         d['article_dst'].append(extras[5:])

#     return ret

In [ ]:
# # remove less than 10 5enc 5 decode for train
# gdd = gdd[gdd['article_id'].apply(lambda x: len(x))>15]
# # create 5 -50+ 
# gdd['article_src'], gdd['article_dst'] = gdd['article_id'].apply(lambda x: x[:5]), gdd['article_id'].apply(lambda x: x[5:])
# del gdd['article_id']
# # split and create record on more than 
# gdd['article_dst'] = gdd.apply(split_row, axis=1)
# df2 = pd.DataFrame.from_dict(d)
# # df2 = df2[df2['article_dst'].apply(lambda x: len(x))>5]
# fdf = pd.concat([df2, gdd])
# del df2
# del gdd

In [ ]:
# fdf['customer_id'] = fdf['customer_id'].astype(int)

In [ ]:
# fdf.dtypes

In [ ]:
# fdf.to_csv("fdf.csv", index=False)
# wandb.save("fdf.csv")
# fdf.head(1000).to_csv("fdf2.csv", index=False)
# wandb.save("fdf2.csv")

In [ ]:
fdf = wandb.restore("fdf.csv", run_path="mayankk-om-dev/HnmRGCNv2/23rwj7i0")

In [ ]:
df = pd.read_csv(fdf.name)
print(df.shape)

In [ ]:
df = df.sample(frac=1, random_state=29)

In [ ]:
df.to_csv("fdf.csv", index=False)
if wmode: wandb.save("fdf.csv")

In [ ]:
from sklearn.model_selection import train_test_split
# traindf, valdf = train_test_split(gd, train_size=0.8, random_state=29)
from torchtext.legacy.data import Field, TabularDataset, BucketIterator

customer = Field(sequential=False, use_vocab=False)
article_src = Field(sequential=True, use_vocab=False, tokenize=lambda x: eval(x))
article_dst = Field(sequential=True, use_vocab=False, pad_token=105540, fix_length=10, tokenize=lambda x: eval(x))

fields = {'customer_id': ('c', customer), 
          'article_src':('asrc', article_src),
          'article_dst':('atrg', article_dst)}




In [ ]:
# fdf.head()

In [ ]:
train = TabularDataset(path='fdf.csv', format='csv', fields=fields)

In [ ]:
train[0].__dict__.keys()

In [ ]:
train[-200].__dict__.values()#105540

In [ ]:
# for batch in train_iterator:
#     customers = batch.c
#     customers = customers.view(1, customers.shape[0])
#     print(customers.shape)
#     break

# Train Model

In [ ]:
from torch.nn.functional import linear, softmax, dropout
import math
from torch.overrides import has_torch_function
Tensor = torch.Tensor
from typing import Callable, List, Optional, Tuple, Union
def _in_projection_packed(
    q: Tensor,
    k: Tensor,
    v: Tensor,
    w: Tensor,
    b: Optional[Tensor] = None,
) -> List[Tensor]:
    r"""
    Performs the in-projection step of the attention operation, using packed weights.
    Output is a triple containing projection tensors for query, key and value.
    Args:
        q, k, v: query, key and value tensors to be projected. For self-attention,
            these are typically the same tensor; for encoder-decoder attention,
            k and v are typically the same tensor. (We take advantage of these
            identities for performance if they are present.) Regardless, q, k and v
            must share a common embedding dimension; otherwise their shapes may vary.
        w: projection weights for q, k and v, packed into a single tensor. Weights
            are packed along dimension 0, in q, k, v order.
        b: optional projection biases for q, k and v, packed into a single tensor
            in q, k, v order.
    Shape:
        Inputs:
        - q: :math:`(..., E)` where E is the embedding dimension
        - k: :math:`(..., E)` where E is the embedding dimension
        - v: :math:`(..., E)` where E is the embedding dimension
        - w: :math:`(E * 3, E)` where E is the embedding dimension
        - b: :math:`E * 3` where E is the embedding dimension
        Output:
        - in output list :math:`[q', k', v']`, each output tensor will have the
            same shape as the corresponding input tensor.
    """
    E = q.size(-1)
    if k is v:
        if q is k:
            # self-attention
            return linear(q, w, b).chunk(3, dim=-1)
        else:
            # encoder-decoder attention
            w_q, w_kv = w.split([E, E * 2])
            if b is None:
                b_q = b_kv = None
            else:
                b_q, b_kv = b.split([E, E * 2])
            return (linear(q, w_q, b_q),) + linear(k, w_kv, b_kv).chunk(2, dim=-1)
    else:
        w_q, w_k, w_v = w.chunk(3)
        if b is None:
            b_q = b_k = b_v = None
        else:
            b_q, b_k, b_v = b.chunk(3)
        return linear(q, w_q, b_q), linear(k, w_k, b_k), linear(v, w_v, b_v)

def _scaled_dot_product_attention(
    q: Tensor,
    k: Tensor,
    v: Tensor,
    attn_mask: Optional[Tensor] = None,
    dropout_p: float = 0.0,
) -> Tuple[Tensor, Tensor]:
    r"""
    Computes scaled dot product attention on query, key and value tensors, using
    an optional attention mask if passed, and applying dropout if a probability
    greater than 0.0 is specified.
    Returns a tensor pair containing attended values and attention weights.
    Args:
        q, k, v: query, key and value tensors. See Shape section for shape details.
        attn_mask: optional tensor containing mask values to be added to calculated
            attention. May be 2D or 3D; see Shape section for details.
        dropout_p: dropout probability. If greater than 0.0, dropout is applied.
    Shape:
        - q: :math:`(B, Nt, E)` where B is batch size, Nt is the target sequence length,
            and E is embedding dimension.
        - key: :math:`(B, Ns, E)` where B is batch size, Ns is the source sequence length,
            and E is embedding dimension.
        - value: :math:`(B, Ns, E)` where B is batch size, Ns is the source sequence length,
            and E is embedding dimension.
        - attn_mask: either a 3D tensor of shape :math:`(B, Nt, Ns)` or a 2D tensor of
            shape :math:`(Nt, Ns)`.
        - Output: attention values have shape :math:`(B, Nt, E)`; attention weights
            have shape :math:`(B, Nt, Ns)`
    """
    B, Nt, E = q.shape
    q = q / math.sqrt(E)
    # (B, Nt, E) x (B, E, Ns) -> (B, Nt, Ns)
    if attn_mask is not None:
        attn = torch.baddbmm(attn_mask, q, k.transpose(-2, -1))
    else:
        attn = torch.bmm(q, k.transpose(-2, -1))
#     print("Before Sftmx", attn)
    attn = softmax(attn, dim=-1)
#     print("After Sftmx", attn)
    if dropout_p > 0.0:
        attn = dropout(attn, p=dropout_p)
    # (B, Nt, Ns) x (B, Ns, E) -> (B, Nt, E)
    output = torch.bmm(attn, v)
    return output, attn

def _in_projection(
    q: Tensor,
    k: Tensor,
    v: Tensor,
    w_q: Tensor,
    w_k: Tensor,
    w_v: Tensor,
    b_q: Optional[Tensor] = None,
    b_k: Optional[Tensor] = None,
    b_v: Optional[Tensor] = None,
) -> Tuple[Tensor, Tensor, Tensor]:
    r"""
    Performs the in-projection step of the attention operation. This is simply
    a triple of linear projections, with shape constraints on the weights which
    ensure embedding dimension uniformity in the projected outputs.
    Output is a triple containing projection tensors for query, key and value.
    Args:
        q, k, v: query, key and value tensors to be projected.
        w_q, w_k, w_v: weights for q, k and v, respectively.
        b_q, b_k, b_v: optional biases for q, k and v, respectively.
    Shape:
        Inputs:
        - q: :math:`(Qdims..., Eq)` where Eq is the query embedding dimension and Qdims are any
            number of leading dimensions.
        - k: :math:`(Kdims..., Ek)` where Ek is the key embedding dimension and Kdims are any
            number of leading dimensions.
        - v: :math:`(Vdims..., Ev)` where Ev is the value embedding dimension and Vdims are any
            number of leading dimensions.
        - w_q: :math:`(Eq, Eq)`
        - w_k: :math:`(Eq, Ek)`
        - w_v: :math:`(Eq, Ev)`
        - b_q: :math:`(Eq)`
        - b_k: :math:`(Eq)`
        - b_v: :math:`(Eq)`
        Output: in output triple :math:`(q', k', v')`,
         - q': :math:`[Qdims..., Eq]`
         - k': :math:`[Kdims..., Eq]`
         - v': :math:`[Vdims..., Eq]`
    """
    Eq, Ek, Ev = q.size(-1), k.size(-1), v.size(-1)
    assert w_q.shape == (Eq, Eq), f"expecting query weights shape of {(Eq, Eq)}, but got {w_q.shape}"
    assert w_k.shape == (Eq, Ek), f"expecting key weights shape of {(Eq, Ek)}, but got {w_k.shape}"
    assert w_v.shape == (Eq, Ev), f"expecting value weights shape of {(Eq, Ev)}, but got {w_v.shape}"
    assert b_q is None or b_q.shape == (Eq,), f"expecting query bias shape of {(Eq,)}, but got {b_q.shape}"
    assert b_k is None or b_k.shape == (Eq,), f"expecting key bias shape of {(Eq,)}, but got {b_k.shape}"
    assert b_v is None or b_v.shape == (Eq,), f"expecting value bias shape of {(Eq,)}, but got {b_v.shape}"
    return linear(q, w_q, b_q), linear(k, w_k, b_k), linear(v, w_v, b_v)

def _mha_shape_check(query: Tensor, key: Tensor, value: Tensor,
                     key_padding_mask: Optional[Tensor], attn_mask: Optional[Tensor], num_heads: int):
    # Verifies the expected shape for `query, `key`, `value`, `key_padding_mask` and `attn_mask`
    # and returns if the input is batched or not.
    # Raises an error if `query` is not 2-D (unbatched) or 3-D (batched) tensor.

    # Shape check.
    if query.dim() == 3:
        # Batched Inputs
        is_batched = True
        assert key.dim() == 3 and value.dim() == 3, \
            ("For batched (3-D) `query`, expected `key` and `value` to be 3-D"
             f" but found {key.dim()}-D and {value.dim()}-D tensors respectively")
        if key_padding_mask is not None:
            assert key_padding_mask.dim() == 2, \
                ("For batched (3-D) `query`, expected `key_padding_mask` to be `None` or 2-D"
                 f" but found {key_padding_mask.dim()}-D tensor instead")
        if attn_mask is not None:
            assert attn_mask.dim() in (2, 3), \
                ("For batched (3-D) `query`, expected `attn_mask` to be `None`, 2-D or 3-D"
                 f" but found {attn_mask.dim()}-D tensor instead")
    elif query.dim() == 2:
        # Unbatched Inputs
        is_batched = False
        assert key.dim() == 2 and value.dim() == 2, \
            ("For unbatched (2-D) `query`, expected `key` and `value` to be 2-D"
             f" but found {key.dim()}-D and {value.dim()}-D tensors respectively")

        if key_padding_mask is not None:
            assert key_padding_mask.dim() == 1, \
                ("For unbatched (2-D) `query`, expected `key_padding_mask` to be `None` or 1-D"
                 f" but found {key_padding_mask.dim()}-D tensor instead")

        if attn_mask is not None:
            assert attn_mask.dim() in (2, 3), \
                ("For unbatched (2-D) `query`, expected `attn_mask` to be `None`, 2-D or 3-D"
                 f" but found {attn_mask.dim()}-D tensor instead")
            if attn_mask.dim() == 3:
                expected_shape = (num_heads, query.shape[0], key.shape[0])
                assert attn_mask.shape == expected_shape, \
                    (f"Expected `attn_mask` shape to be {expected_shape} but got {attn_mask.shape}")
    else:
        raise AssertionError(
            f"query should be unbatched 2D or batched 3D tensor but received {query.dim()}-D query tensor")

    return is_batched

In [ ]:
class RecSTransformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        dim_feedforward,
        dropout,
        max_len_s,
        max_len_t,
        device,
    ):
        super(RecSTransformer, self).__init__()
        weight_a = load_wandb("articleGNNEmb.pt", "mayankk-om-dev/HnmRGCNv2/2oooot9x")
        weight_a = torch.cat([weight_a, torch.Tensor(1, 256)]) # included pad as last token
        # self.article_embedding = nn.Embedding.from_pretrained(weight_a)
        # self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_word_embedding = nn.Embedding.from_pretrained(weight_a) # maximize information for article
        self.src_position_embedding = nn.Embedding(max_len_s, embedding_size)
        # self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        weight_c = load_wandb("customerGNNEmb.pt", "mayankk-om-dev/HnmRGCNv2/2oooot9x")
        # self.trg_word_embedding = nn.Embedding.from_pretrained(weight_c)
        self.trg_word_embedding = self.src_word_embedding # nn.Embedding.from_pretrained(weight_a)#  maximize information for predicting much longer article seq
        self.context_embedding = nn.Embedding.from_pretrained(weight_c)# maximize information for predicting much longer article seq
        self.trg_position_embedding = nn.Embedding(max_len_t, embedding_size)

        self.device = device
        self.max_len_s = max_len_s
        self.max_len_t = max_len_t
        self.transformer = Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            dim_feedforward,
            dropout,
            device=device
        )
        
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)
    
    def forward(self, src, trg, cust):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape
        custID, N = cust.shape
#         print(f"Src {src.shape}, Tar {trg.shape}, Cus {cust.shape}")
        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )
#         print(f"SrcP {src_positions.shape}, TarP {trg_positions.shape}")
#         print(self.src_word_embedding(src).shape)
        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )
#         print("SrcPosition", self.src_position_embedding(src_positions))
#         src_padding_mask = self.make_src_mask(src)
        tgt_key_padding_mask = self.make_src_mask(trg)
#         print(f"embed_src {embed_src}, \n embed_trg {embed_trg}")
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )
        embed_cust = self.context_embedding(cust)
#         print(embed_cust.shape)
        embed_cust = embed_cust.repeat(self.max_len_s,1, 1) #N, seq_len, embs
#         print(f"embed_cust shape {embed_cust.shape}")
        out = self.transformer(
            embed_src,
            embed_trg,
            embed_cust,
#             src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
            tgt_key_padding_mask = tgt_key_padding_mask
        )
        out = self.fc_out(out)
        return out

In [ ]:

def multi_head_attention_forward(
    query: Tensor,
    key: Tensor,
    value: Tensor,
    embed_dim_to_check: int,
    num_heads: int,
    in_proj_weight: Optional[Tensor],
    in_proj_bias: Optional[Tensor],
    bias_k: Optional[Tensor],
    bias_v: Optional[Tensor],
    add_zero_attn: bool,
    dropout_p: float,
    out_proj_weight: Tensor,
    out_proj_bias: Optional[Tensor],
    training: bool = True,
    key_padding_mask: Optional[Tensor] = None,
    need_weights: bool = True,
    attn_mask: Optional[Tensor] = None,
    use_separate_proj_weight: bool = False,
    q_proj_weight: Optional[Tensor] = None,
    k_proj_weight: Optional[Tensor] = None,
    v_proj_weight: Optional[Tensor] = None,
    static_k: Optional[Tensor] = None,
    static_v: Optional[Tensor] = None,
    average_attn_weights: bool = True,
) -> Tuple[Tensor, Optional[Tensor]]:
    r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        embed_dim_to_check: total dimension of the model.
        num_heads: parallel attention heads.
        in_proj_weight, in_proj_bias: input projection weight and bias.
        bias_k, bias_v: bias of the key and value sequences to be added at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        dropout_p: probability of an element to be zeroed.
        out_proj_weight, out_proj_bias: the output projection weight and bias.
        training: apply dropout if is ``True``.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. This is an binary mask. When the value is True,
            the corresponding value on the attention layer will be filled with -inf.
        need_weights: output attn_output_weights.
        attn_mask: 2D or 3D mask that prevents attention to certain positions. A 2D mask will be broadcasted for all
            the batches while a 3D mask allows to specify a different mask for the entries of each batch.
        use_separate_proj_weight: the function accept the proj. weights for query, key,
            and value in different forms. If false, in_proj_weight will be used, which is
            a combination of q_proj_weight, k_proj_weight, v_proj_weight.
        q_proj_weight, k_proj_weight, v_proj_weight, in_proj_bias: input projection weight and bias.
        static_k, static_v: static key and value used for attention operators.
        average_attn_weights: If true, indicates that the returned ``attn_weights`` should be averaged across heads.
            Otherwise, ``attn_weights`` are provided separately per head. Note that this flag only has an effect
            when ``need_weights=True.``. Default: True
    Shape:
        Inputs:
        - query: :math:`(L, E)` or :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, E)` or :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, E)` or :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(S)` or :math:`(N, S)` where N is the batch size, S is the source sequence length.
          If a ByteTensor is provided, the non-zero positions will be ignored while the zero positions
          will be unchanged. If a BoolTensor is provided, the positions with the
          value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.
        - attn_mask: 2D mask :math:`(L, S)` where L is the target sequence length, S is the source sequence length.
          3D mask :math:`(N*num_heads, L, S)` where N is the batch size, L is the target sequence length,
          S is the source sequence length. attn_mask ensures that position i is allowed to attend the unmasked
          positions. If a ByteTensor is provided, the non-zero positions are not allowed to attend
          while the zero positions will be unchanged. If a BoolTensor is provided, positions with ``True``
          are not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
          is provided, it will be added to the attention weight.
        - static_k: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        - static_v: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        Outputs:
        - attn_output: :math:`(L, E)` or :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: Only returned when ``need_weights=True``. If ``average_attn_weights=True``, returns
          attention weights averaged across heads of shape :math:`(L, S)` when input is unbatched or
          :math:`(N, L, S)`, where :math:`N` is the batch size, :math:`L` is the target sequence length, and
          :math:`S` is the source sequence length. If ``average_weights=False``, returns attention weights per
          head of shape :math:`(num_heads, L, S)` when input is unbatched or :math:`(N, num_heads, L, S)`.
    """
    tens_ops = (query, key, value, in_proj_weight, in_proj_bias, bias_k, bias_v, out_proj_weight, out_proj_bias)
    if has_torch_function(tens_ops):
        return handle_torch_function(
            multi_head_attention_forward,
            tens_ops,
            query,
            key,
            value,
            embed_dim_to_check,
            num_heads,
            in_proj_weight,
            in_proj_bias,
            bias_k,
            bias_v,
            add_zero_attn,
            dropout_p,
            out_proj_weight,
            out_proj_bias,
            training=training,
            key_padding_mask=key_padding_mask,
            need_weights=need_weights,
            attn_mask=attn_mask,
            use_separate_proj_weight=use_separate_proj_weight,
            q_proj_weight=q_proj_weight,
            k_proj_weight=k_proj_weight,
            v_proj_weight=v_proj_weight,
            static_k=static_k,
            static_v=static_v,
            average_attn_weights=average_attn_weights,
        )

    is_batched = _mha_shape_check(query, key, value, key_padding_mask, attn_mask, num_heads)

    # For unbatched input, we unsqueeze at the expected batch-dim to pretend that the input
    # is batched, run the computation and before returning squeeze the
    # batch dimension so that the output doesn't carry this temporary batch dimension.
    if not is_batched:
        # unsqueeze if the input is unbatched
        query = query.unsqueeze(1)
        key = key.unsqueeze(1)
        value = value.unsqueeze(1)
        if key_padding_mask is not None:
            key_padding_mask = key_padding_mask.unsqueeze(0)

    # set up shape vars
    tgt_len, bsz, embed_dim = query.shape
    src_len, _, _ = key.shape
    assert embed_dim == embed_dim_to_check, \
        f"was expecting embedding dimension of {embed_dim_to_check}, but got {embed_dim}"
    if isinstance(embed_dim, torch.Tensor):
        # embed_dim can be a tensor when JIT tracing
        head_dim = embed_dim.div(num_heads, rounding_mode='trunc')
    else:
        head_dim = embed_dim // num_heads
    assert head_dim * num_heads == embed_dim, f"embed_dim {embed_dim} not divisible by num_heads {num_heads}"
    if use_separate_proj_weight:
        # allow MHA to have different embedding dimensions when separate projection weights are used
        assert key.shape[:2] == value.shape[:2], \
            f"key's sequence and batch dims {key.shape[:2]} do not match value's {value.shape[:2]}"
    else:
        assert key.shape == value.shape, f"key shape {key.shape} does not match value shape {value.shape}"

    #
    # compute in-projection
    #
    if not use_separate_proj_weight:
        assert in_proj_weight is not None, "use_separate_proj_weight is False but in_proj_weight is None"
        q, k, v = _in_projection_packed(query, key, value, in_proj_weight, in_proj_bias)
    else:
        assert q_proj_weight is not None, "use_separate_proj_weight is True but q_proj_weight is None"
        assert k_proj_weight is not None, "use_separate_proj_weight is True but k_proj_weight is None"
        assert v_proj_weight is not None, "use_separate_proj_weight is True but v_proj_weight is None"
        if in_proj_bias is None:
            b_q = b_k = b_v = None
        else:
            b_q, b_k, b_v = in_proj_bias.chunk(3)
        q, k, v = _in_projection(query, key, value, q_proj_weight, k_proj_weight, v_proj_weight, b_q, b_k, b_v)

    # prep attention mask
    if attn_mask is not None:
        if attn_mask.dtype == torch.uint8:
            warnings.warn("Byte tensor for attn_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead.")
            attn_mask = attn_mask.to(torch.bool)
        else:
            assert attn_mask.is_floating_point() or attn_mask.dtype == torch.bool, \
                f"Only float, byte, and bool types are supported for attn_mask, not {attn_mask.dtype}"
        # ensure attn_mask's dim is 3
        if attn_mask.dim() == 2:
            correct_2d_size = (tgt_len, src_len)
            if attn_mask.shape != correct_2d_size:
                raise RuntimeError(f"The shape of the 2D attn_mask is {attn_mask.shape}, but should be {correct_2d_size}.")
            attn_mask = attn_mask.unsqueeze(0)
        elif attn_mask.dim() == 3:
            correct_3d_size = (bsz * num_heads, tgt_len, src_len)
            if attn_mask.shape != correct_3d_size:
                raise RuntimeError(f"The shape of the 3D attn_mask is {attn_mask.shape}, but should be {correct_3d_size}.")
        else:
            raise RuntimeError(f"attn_mask's dimension {attn_mask.dim()} is not supported")

    # prep key padding mask
    if key_padding_mask is not None and key_padding_mask.dtype == torch.uint8:
        warnings.warn("Byte tensor for key_padding_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead.")
        key_padding_mask = key_padding_mask.to(torch.bool)

    # add bias along batch dimension (currently second)
    if bias_k is not None and bias_v is not None:
        assert static_k is None, "bias cannot be added to static key."
        assert static_v is None, "bias cannot be added to static value."
        k = torch.cat([k, bias_k.repeat(1, bsz, 1)])
        v = torch.cat([v, bias_v.repeat(1, bsz, 1)])
        if attn_mask is not None:
            attn_mask = pad(attn_mask, (0, 1))
            print("Padded Attn_Mask",  attn_mask)
        if key_padding_mask is not None:
            key_padding_mask = pad(key_padding_mask, (0, 1))
    else:
        assert bias_k is None
        assert bias_v is None

    #
    # reshape q, k, v for multihead attention and make em batch first
    #
    q = q.contiguous().view(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    if static_k is None:
        k = k.contiguous().view(k.shape[0], bsz * num_heads, head_dim).transpose(0, 1)
    else:
        # TODO finish disentangling control flow so we don't do in-projections when statics are passed
        assert static_k.size(0) == bsz * num_heads, \
            f"expecting static_k.size(0) of {bsz * num_heads}, but got {static_k.size(0)}"
        assert static_k.size(2) == head_dim, \
            f"expecting static_k.size(2) of {head_dim}, but got {static_k.size(2)}"
        k = static_k
    if static_v is None:
        v = v.contiguous().view(v.shape[0], bsz * num_heads, head_dim).transpose(0, 1)
    else:
        # TODO finish disentangling control flow so we don't do in-projections when statics are passed
        assert static_v.size(0) == bsz * num_heads, \
            f"expecting static_v.size(0) of {bsz * num_heads}, but got {static_v.size(0)}"
        assert static_v.size(2) == head_dim, \
            f"expecting static_v.size(2) of {head_dim}, but got {static_v.size(2)}"
        v = static_v

    # add zero attention along batch dimension (now first)
    if add_zero_attn:
        zero_attn_shape = (bsz * num_heads, 1, head_dim)
        k = torch.cat([k, torch.zeros(zero_attn_shape, dtype=k.dtype, device=k.device)], dim=1)
        v = torch.cat([v, torch.zeros(zero_attn_shape, dtype=v.dtype, device=v.device)], dim=1)
        if attn_mask is not None:
            attn_mask = pad(attn_mask, (0, 1))
        if key_padding_mask is not None:
            key_padding_mask = pad(key_padding_mask, (0, 1))

    # update source sequence length after adjustments
    src_len = k.size(1)

    # merge key padding and attention masks
    if key_padding_mask is not None:
        assert key_padding_mask.shape == (bsz, src_len), \
            f"expecting key_padding_mask shape of {(bsz, src_len)}, but got {key_padding_mask.shape}"
        key_padding_mask = key_padding_mask.view(bsz, 1, 1, src_len).   \
            expand(-1, num_heads, -1, -1).reshape(bsz * num_heads, 1, src_len)
        if attn_mask is None:
            attn_mask = key_padding_mask
        elif attn_mask.dtype == torch.bool:
            attn_mask = attn_mask.logical_or(key_padding_mask)
        else:
            attn_mask = attn_mask.masked_fill(key_padding_mask, float("-1e20"))
    
    # convert mask to float
    if attn_mask is not None and attn_mask.dtype == torch.bool:
        new_attn_mask = torch.zeros_like(attn_mask, dtype=q.dtype)
        new_attn_mask.masked_fill_(attn_mask, float("-1e20"))
        attn_mask = new_attn_mask
#     print("Attention Mask", attn_mask )
    # adjust dropout probability
    if not training:
        dropout_p = 0.0

    #
    # (deep breath) calculate attention and out projection
    #
    attn_output, attn_output_weights = _scaled_dot_product_attention(q, k, v, attn_mask, dropout_p)
    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len * bsz, embed_dim)
    attn_output = linear(attn_output, out_proj_weight, out_proj_bias)
    attn_output = attn_output.view(tgt_len, bsz, attn_output.size(1))

    if need_weights:
        # optionally average attention weights over heads
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        if average_attn_weights:
            attn_output_weights = attn_output_weights.sum(dim=1) / num_heads

        if not is_batched:
            # squeeze the output if input was unbatched
            attn_output = attn_output.squeeze(1)
            attn_output_weights = attn_output_weights.squeeze(0)
        return attn_output, attn_output_weights
    else:
        if not is_batched:
            # squeeze the output if input was unbatched
            attn_output = attn_output.squeeze(1)
        return attn_output, None

In [ ]:
# from torch.nn.quantized.functional import linear

In [ ]:
linear

In [ ]:
import warnings
from typing import Optional, Tuple

import torch
from torch import Tensor
from torch.nn.modules.linear import NonDynamicallyQuantizableLinear
from torch.nn.init import constant_, xavier_normal_, xavier_uniform_
from torch.nn.parameter import Parameter
from torch.nn import Module

class MultiheadAttention(Module):
    r"""Allows the model to jointly attend to information
    from different representation subspaces as described in the paper:
    `Attention Is All You Need <https://arxiv.org/abs/1706.03762>`_.

    Multi-Head Attention is defined as:

    .. math::
        \text{MultiHead}(Q, K, V) = \text{Concat}(head_1,\dots,head_h)W^O

    where :math:`head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)`.

    ``forward()`` will use a special optimized implementation if all of the following
    conditions are met:

    - self attention is being computed (i.e., ``query``, ``key``, and ``value`` are the same tensor. This
      restriction will be loosened in the future.)
    - Either autograd is disabled (using ``torch.inference_mode`` or ``torch.no_grad``) or no tensor argument ``requires_grad``
    - training is disabled (using ``.eval()``)
    - dropout is 0
    - ``add_bias_kv`` is ``False``
    - ``add_zero_attn`` is ``False``
    - ``batch_first`` is ``True`` and the input is batched
    - ``kdim`` and ``vdim`` are equal to ``embed_dim``
    - at most one of ``key_padding_mask`` or ``attn_mask`` is passed
    - if a `NestedTensor <https://pytorch.org/docs/stable/nested.html>`_ is passed, neither ``key_padding_mask``
      nor ``attn_mask`` is passed

    If the optimized implementation is in use, a
    `NestedTensor <https://pytorch.org/docs/stable/nested.html>`_ can be passed for
    ``query``/``key``/``value`` to represent padding more efficiently than using a
    padding mask. In this case, a `NestedTensor <https://pytorch.org/docs/stable/nested.html>`_
    will be returned, and an additional speedup proportional to the fraction of the input
    that is padding can be expected.

    Args:
        embed_dim: Total dimension of the model.
        num_heads: Number of parallel attention heads. Note that ``embed_dim`` will be split
            across ``num_heads`` (i.e. each head will have dimension ``embed_dim // num_heads``).
        dropout: Dropout probability on ``attn_output_weights``. Default: ``0.0`` (no dropout).
        bias: If specified, adds bias to input / output projection layers. Default: ``True``.
        add_bias_kv: If specified, adds bias to the key and value sequences at dim=0. Default: ``False``.
        add_zero_attn: If specified, adds a new batch of zeros to the key and value sequences at dim=1.
            Default: ``False``.
        kdim: Total number of features for keys. Default: ``None`` (uses ``kdim=embed_dim``).
        vdim: Total number of features for values. Default: ``None`` (uses ``vdim=embed_dim``).
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False`` (seq, batch, feature).

    Examples::

        >>> multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        >>> attn_output, attn_output_weights = multihead_attn(query, key, value)

    """
    __constants__ = ['batch_first']
    bias_k: Optional[torch.Tensor]
    bias_v: Optional[torch.Tensor]

    def __init__(self, embed_dim, num_heads, dropout=0., bias=True, add_bias_kv=False, add_zero_attn=False,
                 kdim=None, vdim=None, batch_first=False, device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(MultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.kdim = kdim if kdim is not None else embed_dim
        self.vdim = vdim if vdim is not None else embed_dim
        self._qkv_same_embed_dim = self.kdim == embed_dim and self.vdim == embed_dim

        self.num_heads = num_heads
        self.dropout = dropout
        self.batch_first = batch_first
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        if self._qkv_same_embed_dim is False:
            self.q_proj_weight = Parameter(torch.empty((embed_dim, embed_dim), **factory_kwargs))
            self.k_proj_weight = Parameter(torch.empty((embed_dim, self.kdim), **factory_kwargs))
            self.v_proj_weight = Parameter(torch.empty((embed_dim, self.vdim), **factory_kwargs))
            self.register_parameter('in_proj_weight', None)
        else:
            self.in_proj_weight = Parameter(torch.empty((3 * embed_dim, embed_dim), **factory_kwargs))
            self.register_parameter('q_proj_weight', None)
            self.register_parameter('k_proj_weight', None)
            self.register_parameter('v_proj_weight', None)

        if bias:
            self.in_proj_bias = Parameter(torch.empty(3 * embed_dim, **factory_kwargs))
        else:
            self.register_parameter('in_proj_bias', None)
        self.out_proj = NonDynamicallyQuantizableLinear(embed_dim, embed_dim, bias=bias, **factory_kwargs)

        if add_bias_kv:
            self.bias_k = Parameter(torch.empty((1, 1, embed_dim), **factory_kwargs))
            self.bias_v = Parameter(torch.empty((1, 1, embed_dim), **factory_kwargs))
        else:
            self.bias_k = self.bias_v = None

        self.add_zero_attn = add_zero_attn

        self._reset_parameters()

    def _reset_parameters(self):
        if self._qkv_same_embed_dim:
            xavier_uniform_(self.in_proj_weight)
        else:
            xavier_uniform_(self.q_proj_weight)
            xavier_uniform_(self.k_proj_weight)
            xavier_uniform_(self.v_proj_weight)

        if self.in_proj_bias is not None:
            constant_(self.in_proj_bias, 0.)
            constant_(self.out_proj.bias, 0.)
        if self.bias_k is not None:
            xavier_normal_(self.bias_k)
        if self.bias_v is not None:
            xavier_normal_(self.bias_v)

    def __setstate__(self, state):
        # Support loading old MultiheadAttention checkpoints generated by v1.1.0
        if '_qkv_same_embed_dim' not in state:
            state['_qkv_same_embed_dim'] = True

        super(MultiheadAttention, self).__setstate__(state)
#     (x, tgt_mask, tgt_key_padding_mask)
    def forward(self, query: Tensor, key: Tensor, value: Tensor, key_padding_mask: Optional[Tensor] = None,
                need_weights: bool = True, attn_mask: Optional[Tensor] = None,
                average_attn_weights: bool = True) -> Tuple[Tensor, Optional[Tensor]]:
        r"""
    Args:
        query: Query embeddings of shape :math:`(L, E_q)` for unbatched input, :math:`(L, N, E_q)` when ``batch_first=False``
            or :math:`(N, L, E_q)` when ``batch_first=True``, where :math:`L` is the target sequence length,
            :math:`N` is the batch size, and :math:`E_q` is the query embedding dimension ``embed_dim``.
            Queries are compared against key-value pairs to produce the output.
            See "Attention Is All You Need" for more details.
        key: Key embeddings of shape :math:`(S, E_k)` for unbatched input, :math:`(S, N, E_k)` when ``batch_first=False``
            or :math:`(N, S, E_k)` when ``batch_first=True``, where :math:`S` is the source sequence length,
            :math:`N` is the batch size, and :math:`E_k` is the key embedding dimension ``kdim``.
            See "Attention Is All You Need" for more details.
        value: Value embeddings of shape :math:`(S, E_v)` for unbatched input, :math:`(S, N, E_v)` when
            ``batch_first=False`` or :math:`(N, S, E_v)` when ``batch_first=True``, where :math:`S` is the source
            sequence length, :math:`N` is the batch size, and :math:`E_v` is the value embedding dimension ``vdim``.
            See "Attention Is All You Need" for more details.
        key_padding_mask: If specified, a mask of shape :math:`(N, S)` indicating which elements within ``key``
            to ignore for the purpose of attention (i.e. treat as "padding"). For unbatched `query`, shape should be :math:`(S)`.
            Binary and byte masks are supported.
            For a binary mask, a ``True`` value indicates that the corresponding ``key`` value will be ignored for
            the purpose of attention. For a byte mask, a non-zero value indicates that the corresponding ``key``
            value will be ignored.
        need_weights: If specified, returns ``attn_output_weights`` in addition to ``attn_outputs``.
            Default: ``True``.
        attn_mask: If specified, a 2D or 3D mask preventing attention to certain positions. Must be of shape
            :math:`(L, S)` or :math:`(N\cdot\text{num\_heads}, L, S)`, where :math:`N` is the batch size,
            :math:`L` is the target sequence length, and :math:`S` is the source sequence length. A 2D mask will be
            broadcasted across the batch while a 3D mask allows for a different mask for each entry in the batch.
            Binary, byte, and float masks are supported. For a binary mask, a ``True`` value indicates that the
            corresponding position is not allowed to attend. For a byte mask, a non-zero value indicates that the
            corresponding position is not allowed to attend. For a float mask, the mask values will be added to
            the attention weight.
        average_attn_weights: If true, indicates that the returned ``attn_weights`` should be averaged across
            heads. Otherwise, ``attn_weights`` are provided separately per head. Note that this flag only has an
            effect when ``need_weights=True``. Default: ``True`` (i.e. average weights across heads)

    Outputs:
        - **attn_output** - Attention outputs of shape :math:`(L, E)` when input is unbatched,
          :math:`(L, N, E)` when ``batch_first=False`` or :math:`(N, L, E)` when ``batch_first=True``,
          where :math:`L` is the target sequence length, :math:`N` is the batch size, and :math:`E` is the
          embedding dimension ``embed_dim``.
        - **attn_output_weights** - Only returned when ``need_weights=True``. If ``average_attn_weights=True``,
          returns attention weights averaged across heads of shape :math:`(L, S)` when input is unbatched or
          :math:`(N, L, S)`, where :math:`N` is the batch size, :math:`L` is the target sequence length, and
          :math:`S` is the source sequence length. If ``average_weights=False``, returns attention weights per
          head of shape :math:`(\text{num\_heads}, L, S)` when input is unbatched or :math:`(N, \text{num\_heads}, L, S)`.

        .. note::
            `batch_first` argument is ignored for unbatched inputs.
        """
        is_batched = query.dim() == 3
        why_not_fast_path = ''
        if not is_batched:
            why_not_fast_path = f"input not batched; expected query.dim() of 3 but got {query.dim()}"
        elif query is not key or key is not value:
            # When lifting this restriction, don't forget to either
            # enforce that the dtypes all match or test cases where
            # they don't!
            why_not_fast_path = "non-self attention was used (query, key, and value are not the same Tensor)"
        elif query.dtype != self.in_proj_bias.dtype:
            why_not_fast_path = f"dtypes of query ({query.dtype}) and self.in_proj_bias ({self.in_proj_bias.dtype}) don't match"
        elif query.dtype != self.in_proj_weight.dtype:
            # this case will fail anyway, but at least they'll get a useful error message.
            why_not_fast_path = f"dtypes of query ({query.dtype}) and self.in_proj_weight ({self.in_proj_weight.dtype}) don't match"
        elif self.training:
            why_not_fast_path = "training is enabled"
        elif not self.batch_first:
            why_not_fast_path = "batch_first was not True"
        elif self.bias_k is not None:
            why_not_fast_path = "self.bias_k was not None"
        elif self.bias_v is not None:
            why_not_fast_path = "self.bias_v was not None"
        elif self.dropout:
            why_not_fast_path = f"dropout was {self.dropout}, required zero"
        elif self.add_zero_attn:
            why_not_fast_path = "add_zero_attn was enabled"
        elif not self._qkv_same_embed_dim:
            why_not_fast_path = "_qkv_same_embed_dim was not True"
        elif query.nest.is_nested and (key_padding_mask is not None or attn_mask is not None):
            why_not_fast_path = "key_padding_mask and attn_mask are not supported with NestedTensor input"
        elif not query.nest.is_nested and key_padding_mask is not None and attn_mask is not None:
            why_not_fast_path = "key_padding_mask and attn_mask were both supplied"

        if not why_not_fast_path:
            tensor_args = (
                query,
                key,
                value,
                self.in_proj_weight,
                self.in_proj_bias,
                self.out_proj.weight,
                self.out_proj.bias,
            )
            # We have to use list comprehensions below because TorchScript does not support
            # generator expressions.
            if torch.overrides.has_torch_function(tensor_args):
                why_not_fast_path = "some Tensor argument has_torch_function"
            elif not all([(x.is_cuda or 'cpu' in str(x.device)) for x in tensor_args]):
                why_not_fast_path = "some Tensor argument is neither CUDA nor CPU"
            elif torch.is_grad_enabled() and any([x.requires_grad for x in tensor_args]):
                why_not_fast_path = ("grad is enabled and at least one of query or the "
                                     "input/output projection weights or biases requires_grad")
            if not why_not_fast_path:
                return torch._native_multi_head_attention(
                    query,
                    key,
                    value,
                    self.embed_dim,
                    self.num_heads,
                    self.in_proj_weight,
                    self.in_proj_bias,
                    self.out_proj.weight,
                    self.out_proj.bias,
                    key_padding_mask if key_padding_mask is not None else attn_mask,
                    need_weights,
                    average_attn_weights)
#         any_nested = query.is_nested or key.is_nested or value.is_nested
#         assert not any_nested, ("MultiheadAttention does not support NestedTensor outside of its fast path. " +
#                                 f"The fast path was not hit because {why_not_fast_path}")

        if self.batch_first and is_batched:
            # make sure that the transpose op does not affect the "is" property
            if key is value:
                if query is key:
                    query = key = value = query.transpose(1, 0)
                else:
                    query, key = [x.transpose(1, 0) for x in (query, key)]
                    value = key
            else:
                query, key, value = [x.transpose(1, 0) for x in (query, key, value)]

        if not self._qkv_same_embed_dim:
            attn_output, attn_output_weights = multi_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights,
                attn_mask=attn_mask, use_separate_proj_weight=True,
                q_proj_weight=self.q_proj_weight, k_proj_weight=self.k_proj_weight,
                v_proj_weight=self.v_proj_weight, average_attn_weights=average_attn_weights)
        else:
            attn_output, attn_output_weights = multi_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights,
                attn_mask=attn_mask, average_attn_weights=average_attn_weights)
        if self.batch_first and is_batched:
            return attn_output.transpose(1, 0), attn_output_weights
        else:
            return attn_output, attn_output_weights



In [ ]:
import copy
from typing import Optional, Any, Union, Callable

import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F
from torch.nn import Module, ModuleList, TransformerEncoderLayer, TransformerEncoder, TransformerDecoder
from torch.nn.init import xavier_uniform_
from torch.nn import Dropout, Linear, LayerNorm

class TransformerDecoderLayer(Module):
    r"""
    Args:
        d_model: the number of expected features in the input (required).
        nhead: the number of heads in the multiheadattention models (required).
        dim_feedforward: the dimension of the feedforward network model (default=2048).
        dropout: the dropout value (default=0.1).
        activation: the activation function of the intermediate layer, can be a string
            ("relu" or "gelu") or a unary callable. Default: relu
        layer_norm_eps: the eps value in layer normalization components (default=1e-5).
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False`` (seq, batch, feature).
        norm_first: if ``True``, layer norm is done prior to self attention, multihead
            attention and feedforward operations, respectivaly. Otherwise it's done after.
            Default: ``False`` (after).

    Examples::
        >>> decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
        >>> memory = torch.rand(10, 32, 512)
        >>> tgt = torch.rand(20, 32, 512)
        >>> out = decoder_layer(tgt, memory)

    Alternatively, when ``batch_first`` is ``True``:
        >>> decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8, batch_first=True)
        >>> memory = torch.rand(32, 10, 512)
        >>> tgt = torch.rand(32, 20, 512)
        >>> out = decoder_layer(tgt, memory)
    """
    __constants__ = ['batch_first', 'norm_first']

    def __init__(self, d_model: int, nhead: int, dim_feedforward: int = 2048, dropout: float = 0.1,
                 activation: Union[str, Callable[[Tensor], Tensor]] = F.relu,
                 layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(TransformerDecoderLayer, self).__init__()
        self.self_attn = MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first,
                                            **factory_kwargs)
        self.multihead_attn = MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=batch_first,
                                                 **factory_kwargs)
        # Implementation of Feedforward model
        self.linear1 = Linear(d_model, dim_feedforward, **factory_kwargs)
        self.dropout = Dropout(dropout)
        self.linear2 = Linear(dim_feedforward, d_model, **factory_kwargs)

        self.norm_first = norm_first
        self.norm1 = LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.norm2 = LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.norm3 = LayerNorm(d_model, eps=layer_norm_eps, **factory_kwargs)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)
        self.dropout3 = Dropout(dropout)

        # Legacy string support for activation function.
        if isinstance(activation, str):
            self.activation = _get_activation_fn(activation)
        else:
            self.activation = activation

    def __setstate__(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerDecoderLayer, self).__setstate__(state)

    def forward(self, tgt: Tensor, memory: Tensor, tgt_mask: Optional[Tensor] = None, memory_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None, memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        r"""Pass the inputs (and mask) through the decoder layer.

        Args:
            tgt: the sequence to the decoder layer (required).
            memory: the sequence from the last layer of the encoder (required).
            tgt_mask: the mask for the tgt sequence (optional).
            memory_mask: the mask for the memory sequence (optional).
            tgt_key_padding_mask: the mask for the tgt keys per batch (optional).
            memory_key_padding_mask: the mask for the memory keys per batch (optional).

        Shape:
            see the docs in Transformer class.
        """
        # see Fig. 1 of https://arxiv.org/pdf/2002.04745v1.pdf

        x = tgt
        if self.norm_first:
            x = x + self._sa_block(self.norm1(x), tgt_mask, tgt_key_padding_mask)
            x = x + self._mha_block(self.norm2(x), memory, memory_mask, memory_key_padding_mask)
            x = x + self._ff_block(self.norm3(x))
        else:
            x = self.norm1(x + self._sa_block(x, tgt_mask, tgt_key_padding_mask))
#             print("selfAttention", x)
            x = self.norm2(x + self._mha_block(x, memory, memory_mask, memory_key_padding_mask))
#             print("NormMHA", x)
            x = self.norm3(x + self._ff_block(x))
#             print("Norm3", x)
#         print(x.shape)
        return x


    # self-attention block
    def _sa_block(self, x: Tensor,
                  attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=False)[0]
        return self.dropout1(x)

    # multihead attention block
    def _mha_block(self, x: Tensor, mem: Tensor,
                   attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor]) -> Tensor:
        x = self.multihead_attn(x, mem, mem,
                                attn_mask=attn_mask,
                                key_padding_mask=key_padding_mask,
                                need_weights=False)[0]
        return self.dropout2(x)

    # feed forward block
    def _ff_block(self, x: Tensor) -> Tensor:
        x = self.linear2(self.dropout(self.activation(self.linear1(x))))
        return self.dropout3(x)



def _get_clones(module, N):
    return ModuleList([copy.deepcopy(module) for i in range(N)])


def _get_activation_fn(activation):
    if activation == "relu":
        return F.relu
    elif activation == "gelu":
        return F.gelu

    raise RuntimeError("activation should be relu/gelu, not {}".format(activation))

In [ ]:
# src code modified and piggback

class Transformer(Module):
    def __init__(self, d_model: int = 512, nhead: int = 8, num_encoder_layers: int = 6,
                 num_decoder_layers: int = 6, dim_feedforward: int = 2048, dropout: float = 0.1,
                 activation: Union[str, Callable[[Tensor], Tensor]] = F.relu,
                 custom_encoder: Optional[Any] = None, custom_decoder: Optional[Any] = None,
                 layer_norm_eps: float = 1e-5, batch_first: bool = False, norm_first: bool = False,
                 device=None, dtype=None) -> None:
#         factory_kwargs = {'device': device, 'dtype': dtype}
#         print(factory_kwargs)
        super(Transformer, self).__init__()

        if custom_encoder is not None:
            self.encoder = custom_encoder
        else:
            encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward,
                                                    dropout, device=device, dtype=dtype)
            encoder_norm = LayerNorm(d_model, eps=layer_norm_eps, device=device, dtype=dtype)
            self.encoder = TransformerEncoder(encoder_layer, num_encoder_layers, encoder_norm)

        if custom_decoder is not None:
            self.decoder = custom_decoder
        else:
            decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout,
                                                     device=device, dtype=dtype)                                                    
            decoder_norm = LayerNorm(d_model, eps=layer_norm_eps, device=device, dtype=dtype)
            self.decoder = TransformerDecoder(decoder_layer, num_decoder_layers, decoder_norm)

        self._reset_parameters()

        self.d_model = d_model
        self.nhead = nhead

        self.batch_first = batch_first
        # Adding Context ####
        self.norm = nn.LayerNorm(d_model)

        self.feed_forward = nn.Sequential(
            nn.Linear(2*d_model, dim_feedforward),
            nn.ReLU(),
            nn.Linear(dim_feedforward, d_model),
        )

        self.dropout = nn.Dropout(dropout)
        ### End Add Context #######
        
#     def make_src_mask(self, src):
#         src_mask = src.transpose(0, 1) == 105540

#         # (N, src_len)
#         return src_mask.to(self.device)
    def forward(self, src: Tensor, tgt: Tensor, custD: Tensor, src_mask: Optional[Tensor] = None, tgt_mask: Optional[Tensor] = None,
                memory_mask: Optional[Tensor] = None, src_key_padding_mask: Optional[Tensor] = None,
                tgt_key_padding_mask: Optional[Tensor] = None, memory_key_padding_mask: Optional[Tensor] = None) -> Tensor:
        
        is_batched = src.dim() == 3
        if not self.batch_first and src.size(1) != tgt.size(1) and is_batched:
            raise RuntimeError("the batch number of src and tgt must be equal")
        elif self.batch_first and src.size(0) != tgt.size(0) and is_batched:
            raise RuntimeError("the batch number of src and tgt must be equal")

        if src.size(-1) != self.d_model or tgt.size(-1) != self.d_model:
            raise RuntimeError("the feature number of src and tgt must be equal to d_model")
        if src.size(0) != custD.size(0):
            raise RuntimeError("the number of src and customer must be equal")
#         print("src", src.shape, src)
#         print("tgt", tgt.shape, tgt)
#         print("cust", custD.shape, custD)
#         memory = self.encoder(src, mask=src_mask, src_key_padding_mask=src_key_padding_mask)
#         print(f"memory shape {memory}")
        ### Add context ###############
#         context_custExp = custD #context_cust.unsqueeze(1).repeat(1, self.max_length, 1)
#         context_forward = self.feed_forward(torch.cat([memory, custD], dim=2))
#         print(f"context_forward shape {context_forward}")
#         context_out = self.dropout(self.norm(context_forward + custD)) # still give more importance to customer emb
#         print(f"context_out shape {context_out}")
        context_out = custD #+ memory
#         print("context_out", context_out)
        
        #### End Context ###############
        
#         print("trgtP", tgt_key_padding_mask)
        output = self.decoder(tgt, context_out, tgt_mask=tgt_mask, memory_mask=memory_mask,
                              tgt_key_padding_mask=tgt_key_padding_mask,
                              memory_key_padding_mask=memory_key_padding_mask)
#         print(f"output shape {output}")
        return output

    @staticmethod
    def generate_square_subsequent_mask(sz: int) -> Tensor:
        r"""Generate a square mask for the sequence. The masked positions are filled with float('-inf').
            Unmasked positions are filled with float(0.0).
        """
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

    def _reset_parameters(self):
        r"""Initiate parameters in the transformer model."""

        for p in self.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

In [ ]:
import tqdm
import torch.optim as optim
import time, random
import numpy as np
# import gc
# gc.collect()
# torch.cuda.empty_cache() 


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
save_model = True
torch.manual_seed(99)
torch.backends.cudnn.benchmark = False
# torch.set_deterministic(True)
np.random.seed(99)
random.seed(99)

# Training hyperparameters
num_epochs = 400
learning_rate = 3e-6
batch_size = 64

train_iterator = BucketIterator(
    train, 
    batch_size=batch_size,
    device="cuda"
)

# Model hyperparameters
src_vocab_size = 105540# Number of Articles
trg_vocab_size = 105540# Number of Articles
embedding_size = 256
num_heads = 8
num_encoder_layers = 4
num_decoder_layers = 8
dropout_v = 0.10
max_len_s = 5 # Minimum Article Warm Start for Context
max_len_t = 10 # Maximum Article length to predict
forward_expansion = 2048
src_pad_idx = 105540 # english.vocab.stoi["<pad>"]

# Tensorboard to get nice loss plot
# writer = SummaryWriter("runs/loss_plot")
step = 0


model = RecSTransformer(embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout_v,
    max_len_s,
    max_len_t,
    device).to(device)

# last_model = wandb.restore('0-end-nan.pt', 
#                            run_path="mayankk-om-dev/HnmRGCNv2/3sbjjxyv")

# # use the "name" attribute of the returned object if your framework expects a filename, e.g. as in Keras
# checkpoint = torch.load(last_model.name)

# model.load_state_dict(checkpoint["state_dict"])

# wandb.watch(model)
# print("Successfully Loaded Model")
optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.98),eps=1e-09, lr=learning_rate)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)


# optimizer = ScheduledOptim(
#         optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09),
#         2, 256,4000 )


pad_idx = 105540
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    
    model.train()
    losses = []
    with tqdm.tqdm(train_iterator) as tq:
        for batch_idx, batch in enumerate(tq):
#             print(batch)
            # Get input and targets and get to cuda
            customers = batch.c.to(device)
            article_src = batch.asrc.to(device)
            article_trg = batch.atrg.to(device)
            customers = customers.view(1, customers.shape[0])
#             print(article_trg[-10:, :])
#             print(customers.shape)
            # Forward prop
            output = model(article_src, article_trg[:-1, :], customers)

            # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
            # doesn't take input in that form. For example if we have MNIST we want to have
            # output to be: (N, 10) and targets just (N). Here we can view it in a similar
            # way that we have output_words * batch_size that we want to send in into
            # our cost function, so we need to do some reshapin.
            # Let's also remove the start token while we're at it
            output = output.reshape(-1, output.shape[2])
#             print(output)
            target = article_trg[1:].reshape(-1)
#             print("-"*100)
#             print(target)
#             print("#"*100)
            optimizer.zero_grad()

            loss = criterion(output, target)
#             print("*"*10)
#             print(loss.item())
            losses.append(loss.item())

            # Back prop
            loss.backward()
            # Clip to avoid exploding gradient issues, makes sure grads are
            # within a healthy range
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.98, error_if_nonfinite=False)

            # Gradient descent step
            optimizer.step()
#             optimizer.step_and_update_lr()
            
            tq.set_postfix({'loss': '%.03f' % loss.item()}, refresh=False)
            if wmode: wandb.log({"train_loss_mb": loss.item()})
            # plot to tensorboard
#             writer.add_scalar("Training loss", loss, global_step=step)
            step += 1

        checkpoint = {
                    "state_dict": model.state_dict(),
                    "optimizer": optimizer.state_dict(),
                }
        f_ = f'{epoch}-end-{loss.item()}.pt'

        torch.save(checkpoint, f_)
        lm = f_
        if wmode: 
            wandb.save(f_, policy='now')
        
            if epoch >=5 and epoch % 5==0:
                for fnm in os.listdir():
                    if fnm.endswith('.pt'):
                        if fnm == lm:continue
                        os.remove(fnm)

        mean_loss = sum(losses) / len(losses)
        scheduler.step(mean_loss)

In [ ]:
# transformer_model = Transformer(d_model=256,nhead=8, num_encoder_layers=6)
# src = torch.rand((5,256,32, 100))
# cust = torch.rand((5, 256 ,32,100 ))
# tgt = torch.rand((10, 256, 32,100))
# out = transformer_model(src, tgt, cust)

In [ ]:
# import torch
# import torch.nn as nn

# # Sratch Block with New Architecutre
# class SelfAttention(nn.Module):
#     def __init__(self, embed_size, heads):
#         super(SelfAttention, self).__init__()
#         self.embed_size = embed_size
#         self.heads = heads
#         self.head_dim = embed_size // heads

#         assert (
#             self.head_dim * heads == embed_size
#         ), "Embedding size needs to be divisible by heads"

#         self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
#         self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
#         self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
#         self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

#     def forward(self, values, keys, query, mask):
#         # Get number of training examples
#         N = query.shape[0]

#         value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

#         # Split the embedding into self.heads different pieces
#         values = values.reshape(N, value_len, self.heads, self.head_dim)
#         keys = keys.reshape(N, key_len, self.heads, self.head_dim)
#         query = query.reshape(N, query_len, self.heads, self.head_dim)

#         values = self.values(values)  # (N, value_len, heads, head_dim)
#         keys = self.keys(keys)  # (N, key_len, heads, head_dim)
#         queries = self.queries(query)  # (N, query_len, heads, heads_dim)

#         # Einsum does matrix mult. for query*keys for each training example
#         # with every other training example, don't be confused by einsum
#         # it's just how I like doing matrix multiplication & bmm

#         energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
#         # queries shape: (N, query_len, heads, heads_dim),
#         # keys shape: (N, key_len, heads, heads_dim)
#         # energy: (N, heads, query_len, key_len)

#         # Mask padded indices so their weights become 0
#         if mask is not None:
#             energy = energy.masked_fill(mask == 0, float("-1e20"))

#         # Normalize energy values similarly to seq2seq + attention
#         # so that they sum to 1. Also divide by scaling factor for
#         # better stability
#         attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
#         # attention shape: (N, heads, query_len, key_len)

#         out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
#             N, query_len, self.heads * self.head_dim
#         )
#         # attention shape: (N, heads, query_len, key_len)
#         # values shape: (N, value_len, heads, heads_dim)
#         # out after matrix multiply: (N, query_len, heads, head_dim), then
#         # we reshape and flatten the last two dimensions.

#         out = self.fc_out(out)
#         # Linear layer doesn't modify the shape, final shape will be
#         # (N, query_len, embed_size)

#         return out


# class TransformerBlock(nn.Module):
#     def __init__(self, embed_size, heads, dropout, forward_expansion):
#         super(TransformerBlock, self).__init__()
#         self.attention = SelfAttention(embed_size, heads)
#         self.norm1 = nn.LayerNorm(embed_size)
#         self.norm2 = nn.LayerNorm(embed_size)

#         self.feed_forward = nn.Sequential(
#             nn.Linear(embed_size, forward_expansion * embed_size),
#             nn.ReLU(),
#             nn.Linear(forward_expansion * embed_size, embed_size),
#         )

#         self.dropout = nn.Dropout(dropout)

#     def forward(self, value, key, query, mask):
#         attention = self.attention(value, key, query, mask)

#         # Add skip connection, run through normalization and finally dropout
#         x = self.dropout(self.norm1(attention + query))
#         forward = self.feed_forward(x)
#         out = self.dropout(self.norm2(forward + x))
#         return out


# class Encoder(nn.Module):
#     def __init__(
#         self,
#         src_vocab_size, # article_size for me articles 1--n may only use partial vocab
#         embed_size, #256
#         num_layers,
#         heads,
#         device,
#         forward_expansion,
#         dropout,
#         max_length,
#     ):

#         super(Encoder, self).__init__()
#         self.embed_size = embed_size
#         self.device = device
#         weight_a = load_wandb("articleGNNEmb.pt", "mayankk-om-dev/HnmRGCNv2/2oooot9x")
#         weight_a = torch.cat([weight_a, torch.Tensor(1, 256)]) # included pad as last token
#         self.article_embedding = nn.Embedding.from_pretrained(weight_a)
        
# #         assert weight_c.shape src_vocab_size, embed_size
#         self.position_embedding = nn.Embedding(max_length, embed_size)

#         self.layers = nn.ModuleList(
#             [
#                 TransformerBlock(
#                     embed_size,
#                     heads,
#                     dropout=dropout,
#                     forward_expansion=forward_expansion,
#                 )
#                 for _ in range(num_layers)
#             ]
#         )

#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, mask):
#         N, seq_length = x.shape
#         positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
#         out = self.dropout(
#             (self.article_embedding(x) + self.position_embedding(positions))
#         )

#         # In the Encoder the query, key, value are all the same, it's in the
#         # decoder this will change. This might look a bit odd in this case.
#         for layer in self.layers:
#             out = layer(out, out, out, mask)

#         return out


# class DecoderBlock(nn.Module):
#     def __init__(self, embed_size, heads, forward_expansion, dropout, device):
#         super(DecoderBlock, self).__init__()
#         self.norm = nn.LayerNorm(embed_size)
#         self.attention = SelfAttention(embed_size, heads=heads)
#         self.transformer_block = TransformerBlock(
#             embed_size, heads, dropout, forward_expansion
#         )
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, value, key, src_mask, trg_mask):
#         attention = self.attention(x, x, x, trg_mask)
#         query = self.dropout(self.norm(attention + x))
#         out = self.transformer_block(value, key, query, src_mask)
#         return out


# class Decoder(nn.Module):
#     def __init__(
#         self,
#         trg_vocab_size,
#         embed_size,
#         num_layers,
#         heads,
#         forward_expansion,
#         dropout,
#         device,
#         max_length,
#     ):
#         super(Decoder, self).__init__()
#         self.device = device
# #         self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
#         weight_a = load_wandb("articleGNNEmb.pt", "mayankk-om-dev/HnmRGCNv2/2oooot9x")
#         weight_a = torch.cat([weight_a, torch.Tensor(1, 256)]) # included pad as last token
#         self.article_embedding = nn.Embedding.from_pretrained(weight_a)
        
        
#         self.position_embedding = nn.Embedding(max_length, embed_size)

#         self.layers = nn.ModuleList(
#             [
#                 DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
#                 for _ in range(num_layers)
#             ]
#         )
#         self.fc_out = nn.Linear(embed_size, trg_vocab_size)
#         self.dropout = nn.Dropout(dropout)

#     def forward(self, x, enc_out, src_mask, trg_mask):
#         N, seq_length = x.shape
#         positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
#         x = self.dropout((self.article_embedding(x) + self.position_embedding(positions)))

#         for layer in self.layers:
#             x = layer(x, enc_out, enc_out, src_mask, trg_mask)

#         out = self.fc_out(x)

#         return out


# class VTransformer(nn.Module):
#     def __init__(
#         self,
#         src_vocab_size,
#         trg_vocab_size,
#         src_pad_idx,
#         trg_pad_idx,
#         embed_size=256,
#         num_layers=6,
#         forward_expansion=4,
#         heads=8,
#         dropout=0,
#         device="cpu",
#         max_ls=5,
#         max_lt=10,
#     ):

#         super(VTransformer, self).__init__()

#         self.encoder = Encoder(
#             src_vocab_size,
#             embed_size,
#             num_layers,
#             heads,
#             device,
#             forward_expansion,
#             dropout,
#             max_ls,
#         )

#         self.decoder = Decoder(
#             trg_vocab_size,
#             embed_size,
#             num_layers,
#             heads,
#             forward_expansion,
#             dropout,
#             device,
#             max_lt,
#         )
#         # Adding Context ####
# #         self.norm = nn.LayerNorm(embed_size)

# #         self.feed_forward = nn.Sequential(
# #             nn.Linear(2*embed_size, forward_expansion *2* embed_size),
# #             nn.ReLU(),
# #             nn.Linear(forward_expansion * 2* embed_size, embed_size),
# #         )

# #         self.dropout = nn.Dropout(dropout)
#         weight_c = load_wandb("customerGNNEmb.pt", "mayankk-om-dev/HnmRGCNv2/2oooot9x")
#         self.context_embedding = nn.Embedding.from_pretrained(weight_c)
        
#         ### End Add Context #######
#         self.src_pad_idx = src_pad_idx
#         self.trg_pad_idx = trg_pad_idx
#         self.device = device
#         self.max_ls = max_ls
#         self.max_lt = max_lt

#     def make_src_mask(self, src):
#         src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
#         # (N, 1, 1, src_len)
#         return src_mask.to(self.device)

#     def make_trg_mask(self, trg):
#         N, trg_len = trg.shape
#         trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
#             N, 1, trg_len, trg_len
#         )

#         return trg_mask.to(self.device)

#     def forward(self, src, trg, customer): # how to pass customer
#         src_mask = self.make_src_mask(src)
#         trg_mask = self.make_trg_mask(trg)
#         enc_src = self.encoder(src, src_mask)
#         print(enc_src.shape)
#         # customer emb N,
# #         context_cust = self.customer_embedding(customer) # N, emb_size_c
# #         context_custExp = context_cust.unsqueeze(1).repeat(1, self.max_length, 1)
# #         # somehow reshape to N, query_len, emb_size_c (repeat)
# #         context_forward = self.feed_forward(torch.concat([enc_src, context_custExp], dim=2))
# #         context_out = self.dropout(self.norm(context_forward + context_cust)) # still give more importance to customer emb
        
#         embed_cust = self.context_embedding(cust)
#         embed_cust = embed_cust.repeat(self.max_ls,1, 1) #N, seq_len, embs
#         print(embed_cust.shape)
#         context_out = embed_cust + enc_src
        
#         out = self.decoder(trg, context_out, src_mask, trg_mask)
#         return out



In [ ]:
# import tqdm
# import torch.optim as optim
# import time, random
# import numpy as np
# # import gc
# # gc.collect()
# # torch.cuda.empty_cache() 


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # device = torch.device("cpu")
# save_model = True
# torch.manual_seed(99)
# torch.backends.cudnn.benchmark = False
# # torch.set_deterministic(True)
# np.random.seed(99)
# random.seed(99)

# # Training hyperparameters
# num_epochs = 400
# learning_rate = 3e-4
# batch_size = 32

# train_iterator = BucketIterator(
#     train, 
#     batch_size=batch_size,
#     device="cuda"
# )

# # Model hyperparameters
# src_vocab_size = 105540# Number of Articles
# trg_vocab_size = 105540# Number of Articles
# embedding_size = 256
# num_heads = 8
# num_encoder_layers = 4
# num_decoder_layers = 8
# dropout = 0.10
# max_len_s = 5 # Minimum Article Warm Start for Context
# max_len_t = 10 # Maximum Article length to predict
# forward_expansion = 2048
# src_pad_idx = 105540 # english.vocab.stoi["<pad>"]
# trg_pad_idx = 105540 
# # Tensorboard to get nice loss plot
# # writer = SummaryWriter("runs/loss_plot")
# step = 0


# model = VTransformer(src_vocab_size,
#         trg_vocab_size,
#         src_pad_idx,
#         trg_pad_idx,
#         embed_size=256,
#         num_layers=6,
#         forward_expansion=4,
#         heads=8,
#         dropout=0.10,
#         device=device,
#         max_ls=5,
#         max_lt=10,).to(device)

# # last_model = wandb.restore('0-end-nan.pt', 
# #                            run_path="mayankk-om-dev/HnmRGCNv2/3sbjjxyv")

# # # use the "name" attribute of the returned object if your framework expects a filename, e.g. as in Keras
# # checkpoint = torch.load(last_model.name)

# # model.load_state_dict(checkpoint["state_dict"])

# # wandb.watch(model)
# # print("Successfully Loaded Model")
# optimizer = optim.Adam(model.parameters(), betas=(0.9, 0.98),eps=1e-09, lr=learning_rate)

# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, factor=0.1, patience=10, verbose=True
# )


# # optimizer = ScheduledOptim(
# #         optim.Adam(model.parameters(), betas=(0.9, 0.98), eps=1e-09),
# #         2, 256,4000 )


# pad_idx = 105540
# criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

# for epoch in range(num_epochs):
#     print(f"[Epoch {epoch} / {num_epochs}]")

    
#     model.train()
#     losses = []
#     with tqdm.tqdm(train_iterator) as tq:
#         for batch_idx, batch in enumerate(tq):
# #             print(batch)
#             # Get input and targets and get to cuda
#             customers = batch.c.to(device)
#             article_src = batch.asrc.to(device)
#             article_trg = batch.atrg.to(device)
#             customers = customers.view(1, customers.shape[0])
# #             print(article_trg[-10:, :])
# #             print(customers.shape)
#             # Forward prop
#             output = model(article_src, article_trg[:-1, :], customers)

#             # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
#             # doesn't take input in that form. For example if we have MNIST we want to have
#             # output to be: (N, 10) and targets just (N). Here we can view it in a similar
#             # way that we have output_words * batch_size that we want to send in into
#             # our cost function, so we need to do some reshapin.
#             # Let's also remove the start token while we're at it
#             output = output.reshape(-1, output.shape[2])
#             print(output)
#             target = article_trg[1:].reshape(-1)
#             print("-"*100)
#             print(target)
#             print("#"*100)
#             optimizer.zero_grad()

#             loss = criterion(output, target)
#             print("*"*10)
#             print(loss.item())
#             losses.append(loss.item())

#             # Back prop
#             loss.backward()
#             # Clip to avoid exploding gradient issues, makes sure grads are
#             # within a healthy range
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.98, error_if_nonfinite=False)

#             # Gradient descent step
#             optimizer.step()
# #             optimizer.step_and_update_lr()
            
#             tq.set_postfix({'loss': '%.03f' % loss.item()}, refresh=False)
# #             wandb.log({"train_loss_mb": loss.item()})
#             # plot to tensorboard
# #             writer.add_scalar("Training loss", loss, global_step=step)
#             step += 1

#         checkpoint = {
#                     "state_dict": model.state_dict(),
#                     "optimizer": optimizer.state_dict(),
#                 }
#         f_ = f'{epoch}-end-{loss.item()}.pt'

#         torch.save(checkpoint, f_)
#         lm = f_
# #         wandb.save(f_, policy='now')
#         if epoch >=5 and epoch % 5==0:
#             for fnm in os.listdir():
#                 if fnm.endswith('.pt'):
#                     if fnm == lm:continue
#                     os.remove(fnm)
        
#         mean_loss = sum(losses) / len(losses)
#         scheduler.step(mean_loss)

In [ ]:
# '''A wrapper class for scheduled optimizer '''
# import numpy as np

# class ScheduledOptim():
#     '''A simple wrapper class for learning rate scheduling'''

#     def __init__(self, optimizer, lr_mul, d_model, n_warmup_steps):
#         self._optimizer = optimizer
#         self.lr_mul = lr_mul
#         self.d_model = d_model
#         self.n_warmup_steps = n_warmup_steps
#         self.n_steps = 0


#     def step_and_update_lr(self):
#         "Step with the inner optimizer"
#         self._update_learning_rate()
#         self._optimizer.step()


#     def zero_grad(self):
#         "Zero out the gradients with the inner optimizer"
#         self._optimizer.zero_grad()


#     def _get_lr_scale(self):
#         d_model = self.d_model
#         n_steps, n_warmup_steps = self.n_steps, self.n_warmup_steps
#         return (d_model ** -0.5) * min(n_steps ** (-0.5), n_steps * n_warmup_steps ** (-1.5))


#     def _update_learning_rate(self):
#         ''' Learning rate scheduling per step '''

#         self.n_steps += 1
#         lr = self.lr_mul * self._get_lr_scale()

#         for param_group in self._optimizer.param_groups:
#             param_group['lr'] = lr